# Interactive Treemap Visualization 

This notebook can visualize export treemap interactively. A user can specify the country, year, quality and amount thresholds to control what comes up in the treemap. The file `complete.csv` is output from the original notebook template and contains the data needed. Requires vistk, pandas, seaborn and ipywidgets. 

In [1]:
import sys
sys.path.append("./modules")
import vistk
import pandas as pd
import seaborn as sns
import numpy as np

In [2]:
from ipywidgets import interact
from ipywidgets import widgets

In [ ]:
##inputText = widgets.Text()

##def takename(sender):
##    filename=inputText.value

##inputText.on_submit(takename)
##inputText

In [ ]:
##filename

In [3]:
# Load complete.csv into a dataframe.
completedf=pd.read_csv('complete.csv')
completedf=completedf.drop("Unnamed: 0", 1)
completedf.head()

,year,iso3,sitc4,exports,quality_exp,imports,quality_imp,code,color,community_id,name,category,continent,country,subcontinent
0,1984,DOM,11,NaN,NaN,52.08442,0.702034,11,#ffe999,81,Live bovines,0,Americas,Dominican Republic,Caribbean
1,1984,JOR,7264,NaN,NaN,882.23070,1.248837,7264,#9edae5,10,Printing presses,7,Asia,Hashemite Kingdom of Jordan,Western Asia
2,1984,MLT,7264,33.6984,1.1567,338.30020,0.937385,7264,#9edae5,10,Printing presses,7,Europe,Republic of Malta,Southern Europe
3,1984,BGD,7264,NaN,NaN,1146.31800,0.902277,7264,#9edae5,10,Printing presses,7,Asia,People's Republic of Bangladesh,Southern Asia
4,1984,GHA,7264,NaN,NaN,31.46410,1.159864,7264,#9edae5,10,Printing presses,7,Africa,Republic of Ghana,Western Africa


In [4]:
# Get a list of all unqiue country names. 
allcountries=[e.decode('utf-8') for e in list(set(completedf['country']))]
allcountries.sort()

#### Making a dictionary of sitc4 and product name

In [5]:
# Getting a dictionary of product names keyed by sitc4 code
key={}
for r in completedf.itertuples():
    key[r[3]]=r[11]

In [6]:
# Getting a list of unique sitc4 code
sitc4=[e for e in key.keys()]
sitc4.sort()

In [7]:
# Getting a list of unique product names
products=[e for e in key.values()] # Need to find a better way to drop NaN than this.
products.sort()
products=products[134:]

#### A bunch of helper functions that will be used for making the treemap

In [8]:
# Create a range of colors based on the length of spectrum, which specifies how many colors are needed.
def make_color(spectrum):
    colors=sns.color_palette("Blues", spectrum)
    colors=['#%02x%02x%02x' % (a*256, b*256, c*256) for (a, b, c) in colors]
    return colors

In [9]:
# Sorting and Adding a column of color values to the input dafaframe
def sort_color(df, field='quality_exp'):
    df=df.sort_values(by=field)
    df[field+'_color']=make_color(len(df))
    return df

In [10]:
# Filtering the data based on amount and quality threshold
def threshold(df, quality_threshold=0.0, amount_threshold=0.0):
    df=df[(df['quality_exp']>quality_threshold) & (df['exports']>amount_threshold)]
    return df

In [11]:
# Defining the function that is called to make the treemap
def make_product_treemap(country, year, quality_threshold, amount_threshold):
    df=completedf[(completedf['country']==country) & (completedf['year']==year)].dropna(subset=['exports', 'quality_exp'])
    df=threshold(df, quality_threshold, amount_threshold)
    df=sort_color(df)
    title='Export of %s in %s' % (country, year)
    treemap=vistk.Treemap(id='sitc4', color='quality_exp_color', name='name', size='exports', sort='exports',
                            year=year, title=title)
    treemap.draw(df)

In [12]:
# Defining the function that is called to make the treemap
def make_country_treemap(product, year, quality_threshold, amount_threshold):
    df=completedf[(completedf['name']==product) & (completedf['year']==year)].dropna(subset=['exports', 'quality_exp'])
    df=threshold(df, quality_threshold, amount_threshold)
    df=sort_color(df)
    title='Export of %s in %s' % (product, year)
    treemap=vistk.Treemap(id='iso3', color='quality_exp_color', name='country', size='exports', sort='exports',
                            year=year, title=title)
    treemap.draw(df)

## Visualizing Export Treemap (by Country)

In [13]:
# Let's see it! 
interact(make_product_treemap, country=allcountries, year=(1990, 2000, 1), 
         quality_threshold=(0.1, 4.0, 0.1), amount_threshold=(0, 10000, 100))

<IPython.core.display.Javascript object>

## Visualizing Export Treemap (by Product)

In [14]:
# Let's see it! 
interact(make_country_treemap, product=products, year=(1990, 2000, 1), 
         quality_threshold=(0.1, 4.0, 0.1), amount_threshold=(0, 10000, 100))

<IPython.core.display.Javascript object>